In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

In [ ]:
# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyter_loader import JupyterLoader

loader = JupyterLoader()
loader.load_all()

# Simplified Evaluation Function

The Simplified Evaluation Function uses a different set of piece values.
They base unit used here is centipawn, so a pawn is by definition valued with 100.
The other values are using the accuracy of the centipawn unit to assign slightly more accurate values to the pieces.

In [ ]:
import chess

SIMPLIFIED_EVALUATION_PIECE_VALUES = {
    chess.PAWN: 100,
    chess.KNIGHT: 320,
    chess.BISHOP: 330,
    chess.ROOK: 500,
    chess.QUEEN: 900,
    chess.KING: 20000
}

SIMPLIFIED_EVALUATION_VALUE_CHECKMATE = 30000
SIMPLIFIED_EVALUATION_VALUE_DRAW = 0

The second idea that the Simplified Evaluation Engine is built on is [Piece Square Tables](https://www.chessprogramming.org/Piece-Square_Tables). 
In order for the engine to strategically place its pieces, well-placed pieces are additionally rewarded and poorly-placed pieces are penalized. 
This is technically realized by assigning a specific value to each square on the chessboard for each type of piece. 
`SIMPLIFIED_EVALUATION_PAWN_SQUARE_VALUES` shows the values of the chess squares as a 2 dimensional 8 x 8 list for the white pawn. 
The first entry corresponds to the last row on the chess board from the player's perspective and the last entry to the first row. 
You can see, for example, that pawns on the seventh row for White and second row for black receive a high bonus of 50 because they are about to be promoted.

In [ ]:
SIMPLIFIED_EVALUATION_PAWN_SQUARE_VALUES = [
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [50, 50, 50, 50, 50, 50, 50, 50],
    [10, 10, 20, 30, 30, 20, 10, 10],
    [ 5,  5, 10, 25, 25, 10,  5,  5],
    [ 0,  0,  0, 20, 20,  0,  0,  0],
    [ 5, -5,-10,  0,  0,-10, -5,  5],
    [ 5, 10, 10,-20,-20, 10, 10,  5],
    [ 0,  0,  0,  0,  0,  0,  0,  0]
] # yapf: disable

In order to simplify the calculation later, the figure values defined above can be written directly into the table as an offset. 
This is done by the helper function `add_offset_to_square_values`, which takes a piece square table `square_values` and an offset `offset` as parameters and then adds the offset to each entry in the table.

In [ ]:
def add_offset_to_square_values(square_values: list[list[int]], offset: int):
    for row in range(8):
        for column in range(8):
            square_values[row][column] += offset

For the `SIMPLIFIED_EVALUATION_PAWN_SQUARE_VALUES` the offset of the piece value of a pawn from the `SIMPLIFIED_EVALUATION_PIECE_VALUES` dictionary is added.

In [ ]:
add_offset_to_square_values(
    SIMPLIFIED_EVALUATION_PAWN_SQUARE_VALUES,
    SIMPLIFIED_EVALUATION_PIECE_VALUES[chess.PAWN]
)

These tables are defined accordingly for all pieces
and the piece values are added to the tables as done with the `SIMPLIFIED_EVALUATION_PAWN_SQUARE_VALUES`.
An even more advanced technique is to use different table for black and white, but with the `Simplified Evaluation Function` the same tables are used for both players.

In [ ]:
SIMPLIFIED_EVALUATION_KNIGHT_SQUARE_VALUES = [
    [-50,-40,-30,-30,-30,-30,-40,-50],
    [-40,-20,  0,  0,  0,  0,-20,-40],
    [-30,  0, 10, 15, 15, 10,  0,-30],
    [-30,  5, 15, 20, 20, 15,  5,-30],
    [-30,  0, 15, 20, 20, 15,  0,-30],
    [-30,  5, 10, 15, 15, 10,  5,-30],
    [-40,-20,  0,  5,  5,  0,-20,-40],
    [-50,-40,-30,-30,-30,-30,-40,-50]
]  # yapf: disable

add_offset_to_square_values(
    SIMPLIFIED_EVALUATION_KNIGHT_SQUARE_VALUES,
    SIMPLIFIED_EVALUATION_PIECE_VALUES[chess.KNIGHT]
)

In [ ]:
SIMPLIFIED_EVALUATION_BISHOP_SQUARE_VALUES = [
    [-20,-10,-10,-10,-10,-10,-10,-20],
    [-10,  0,  0,  0,  0,  0,  0,-10],
    [-10,  0,  5, 10, 10,  5,  0,-10],
    [-10,  5,  5, 10, 10,  5,  5,-10],
    [-10,  0, 10, 10, 10, 10,  0,-10],
    [-10, 10, 10, 10, 10, 10, 10,-10],
    [-10,  5,  0,  0,  0,  0,  5,-10],
    [-20,-10,-10,-10,-10,-10,-10,-20]
]  # yapf: disable

add_offset_to_square_values(
    SIMPLIFIED_EVALUATION_BISHOP_SQUARE_VALUES,
    SIMPLIFIED_EVALUATION_PIECE_VALUES[chess.BISHOP]
)

In [ ]:
SIMPLIFIED_EVALUATION_ROOK_SQUARE_VALUES = [
    [ 0,  0,  0,  0,  0,  0,  0,  0],
    [ 5, 10, 10, 10, 10, 10, 10,  5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [-5,  0,  0,  0,  0,  0,  0, -5],
    [ 0,  0,  0,  5,  5,  0,  0,  0]
] # yapf: disable

add_offset_to_square_values(
    SIMPLIFIED_EVALUATION_ROOK_SQUARE_VALUES,
    SIMPLIFIED_EVALUATION_PIECE_VALUES[chess.ROOK]
)

In [ ]:
SIMPLIFIED_EVALUATION_QUEEN_SQUARE_VALUES = [
    [-20,-10,-10, -5, -5,-10,-10,-20],
    [-10,  0,  0,  0,  0,  0,  0,-10],
    [-10,  0,  5,  5,  5,  5,  0,-10],
    [ -5,  0,  5,  5,  5,  5,  0, -5],
    [  0,  0,  5,  5,  5,  5,  0, -5],
    [-10,  5,  5,  5,  5,  5,  0,-10],
    [-10,  0,  5,  0,  0,  0,  0,-10],
    [-20,-10,-10, -5, -5,-10,-10,-20]
] # yapf: disable

add_offset_to_square_values(
    SIMPLIFIED_EVALUATION_QUEEN_SQUARE_VALUES,
    SIMPLIFIED_EVALUATION_PIECE_VALUES[chess.QUEEN]
)

In [ ]:
SIMPLIFIED_EVALUATION_KING_SQUARE_VALUES = [
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-30,-40,-40,-50,-50,-40,-40,-30],
    [-20,-30,-30,-40,-40,-30,-30,-20],
    [-10,-20,-20,-20,-20,-20,-20,-10],
    [ 20, 20,  0,  0,  0,  0, 20, 20],
    [ 20, 30, 10,  0,  0, 10, 30, 20],
]# yapf: disable

add_offset_to_square_values(
    SIMPLIFIED_EVALUATION_KING_SQUARE_VALUES,
    SIMPLIFIED_EVALUATION_PIECE_VALUES[chess.KING]
)

After defining all piece square tables,
a dictionary `SIMPLIFIED_EVALUATION_PIECE_SQUARE_VALUES` can be created
mapping the piece to the corresponding table
as was done for the piece values.

In [ ]:
# TODO: Add support for endgame square table (for instance for the king)
# Idea: Two tables (beginning and end) -> interpolate (https://www.chessprogramming.org/Piece-Square_Tables#Multiple_Tables)
SIMPLIFIED_EVALUATION_PIECE_SQUARE_VALUES = {
    chess.PAWN: SIMPLIFIED_EVALUATION_PAWN_SQUARE_VALUES,
    chess.KNIGHT: SIMPLIFIED_EVALUATION_KNIGHT_SQUARE_VALUES,
    chess.BISHOP: SIMPLIFIED_EVALUATION_BISHOP_SQUARE_VALUES,
    chess.ROOK: SIMPLIFIED_EVALUATION_ROOK_SQUARE_VALUES,
    chess.QUEEN: SIMPLIFIED_EVALUATION_QUEEN_SQUARE_VALUES,
    chess.KING: SIMPLIFIED_EVALUATION_KING_SQUARE_VALUES
}

We will now create a new class of `Evaluator` the `PieceSquareEvaluator`
capable of using the piece square tables to calculate the score.
It expects at creation time:
- a dictionary of the piece values
- a dictionary of the piece square tables
- a checkmate value
- a draw value

In [ ]:
from converted_notebooks.s08_evaluation import Evaluator


class PieceSquareEvaluator(Evaluator):

    def __init__(
        self,
        piece_values,
        piece_square_values,
        value_checkmate: int,
        value_draw: int
    ):
        super().__init__(value_checkmate, value_draw)
        self.piece_values = piece_values
        self.piece_square_values = piece_square_values

The method `_get_piece_square_value` takes a chess piece `piece` and its position split in `file` and `rank` as parameters 
and returns the value from the corresponding piece square table.
For white the table needs to be mirrored at the x-axis;
this is done by counting from the other side by accessing `-(rank+1)`.

In [ ]:
def _get_piece_square_value(self, piece: chess.Piece, file: int, rank: int):
    if piece.color == chess.WHITE:
        return self.piece_square_values[piece.piece_type][-(rank + 1)][file]
    else:
        return self.piece_square_values[piece.piece_type][rank][file]


PieceSquareEvaluator._get_piece_square_value = _get_piece_square_value

As iterating on chess board squares is done frequently,
another helper function `_get_square_value` is implemented.
It takes the chess board `board` and a chess board square `square` as parameters, 
calculates the piece, and the file, and the rank,
and returns the result of calling `_get_piece_square_value` with those parameters.

In [ ]:
def _get_square_value(self, board: chess.Board, square: chess.Square):
    piece = board.piece_at(square)
    file = chess.square_file(square)
    rank = chess.square_rank(square)
    return self._get_piece_square_value(piece, file, rank)


PieceSquareEvaluator._get_square_value = _get_square_value

Now the method `heuristic` can be defined, which works similary as before.
It takes a chess board `board` as parameter and returns the evaluation from the current players point of view.
To do so it sums the values of all squares occupied by the current player up.
Afterwards it subtracts the sum of all occupied fields of the oppenent.

In [ ]:
def heuristic(self, board: chess.Board) -> int:
    score = 0

    for square in chess.scan_reversed(board.occupied_co[board.turn]):
        score += self._get_square_value(board, square)

    for square in chess.scan_reversed(board.occupied_co[not board.turn]):
        score -= self._get_square_value(board, square)

    return score


PieceSquareEvaluator.heuristic = heuristic

After defining the `PieceSquareEvaluator` class
it can now be instantiated using the settings corresponding to the simplified evaluation function.

In [ ]:
simplified_evaluator = PieceSquareEvaluator(
    piece_values=SIMPLIFIED_EVALUATION_PIECE_VALUES,
    piece_square_values=SIMPLIFIED_EVALUATION_PIECE_SQUARE_VALUES,
    value_checkmate=SIMPLIFIED_EVALUATION_VALUE_CHECKMATE,
    value_draw=SIMPLIFIED_EVALUATION_VALUE_DRAW
)

The `simplified_evaluator` can be tested in a similiar way to the previous evaluators by creating an engine using it,
but its results can't be compared to the previous engines results,
as the evaluation function is an entirely different one.

In [ ]:
import random
from converted_notebooks.s09_minimax_engine import middlegame_board
from converted_notebooks.s11_iterative_deepening import IterativeAlphaBetaCached

random.seed(42)

engine = IterativeAlphaBetaCached(
    evaluator=simplified_evaluator, max_look_ahead_depth=3
)
result_simplified_evaluation = engine.analyse(middlegame_board)

The jupyter notebook extension `line_profiler` defines the magic command `lprun` to analyse the performance of a command.
In this case it useful to see 
which instruction is called the most and which is taking the most time
when running an analysis of the board with `engine.analyse(middlegame_board)`. The most intersting function is the `_value` function as this is where the recursive algorithm happens. There is some knowledge needed to find the exact name of the function as we have used inheritance and decorators.

In [ ]:
from converted_notebooks.s10_alpha_beta_engine import AlphaBetaEngine

%load_ext line_profiler
random.seed(42)

engine = IterativeAlphaBetaCached(
    evaluator=simplified_evaluator, max_look_ahead_depth=4
)
%lprun -f AlphaBetaEngine._value engine.analyse(middlegame_board)

The evaluation shows that all calls of the heuristic with `self.evaluator.heuristic(board)` account for about 50% of the total runtime. In second and third place with approximatly 15% is the evaluation `self.evaluator.evaluate(board)` and the call of `board.push(move)`. For these, no direct optimizations are possible, since for this the implementation of the `python-chess` library would have to be optimized. 
It is possible though to speed up the heuristic by calculating the score incrementally.

For the incremental implementation, some helper classes need to be defined first. First we create a class `PostionedPiece`, which is a wrapper around the `chess.piece` class. In contrast to the original class, this one captures informations about the board context. It therefore saves the file and the rank of the piece and for ease of use the constructes square.
The class only has a constructor, which taktes the piece `piece` and the file `file` and rank `rank` of the piece as parameters and saves these as attributes.

In [ ]:
class PostionedPiece:

    def __init__(self, piece: chess.Piece, file: int, rank: int):
        self.piece = piece
        self.file = file
        self.rank = rank
        self.square = chess.square(file_index=file, rank_index=rank)

Next, we define a class `DetailedMove` which is as the name suggests a wrapper around the `chess.Move` class. Similary to before this class contains context from the board and does some precalculations. The most important function is the constructor, which takes the chess board `board` and the move `move` as parameters. Then it first constructs the piece that moved as a `PositionedPiece`. The same is done for the piece placed at the end of the move. This is usually the same piece as the moving piece, except when there is a promotion. In that case the piece type changes. In the last step the captured piece if any is constructed. There is one special case that needs to be handeled, the en passant capture.

The class also defines a method `has_capture` which has no parameter and simply returns a boolean indicating if the move captured any piece. 

In [ ]:
class DetailedMove:

    def __init__(self, board: chess.Board, move: chess.Move):
        self.move = move

        self.movedPiece = PostionedPiece(
            piece=board.piece_at(move.from_square),
            file=chess.square_file(move.from_square),
            rank=chess.square_rank(move.from_square)
        )

        if move.promotion:
            self.placedPiece = PostionedPiece(
                piece=chess.Piece(move.promotion, self.movedPiece.piece.color),
                file=chess.square_file(move.to_square),
                rank=chess.square_rank(move.to_square)
            )
        else:
            self.placedPiece = PostionedPiece(
                piece=self.movedPiece.piece,
                file=chess.square_file(move.to_square),
                rank=chess.square_rank(move.to_square)
            )

        if board.is_en_passant(move):
            self.capturedPiece = PostionedPiece(
                piece=chess.Piece(chess.PAWN, not board.turn),
                file=self.movedPiece.file,
                rank=self.placedPiece.rank
            )
        elif (piece := board.piece_at(move.to_square)) is not None:
            self.capturedPiece = PostionedPiece(
                piece=piece,
                file=self.placedPiece.file,
                rank=self.placedPiece.rank
            )
        else:
            self.capturedPiece = None

    def has_capture(self):
        return self.capturedPiece is not None

Before we implement the simplified evaluation incrementally, we first define another general interface `IncrementalEvaluator` that inherits from `Evaluator`. It adds four additional methods which concrete subclasses have to implement. The first function `init` takes the current board `board` as a parameter and will be called at the beginning of an evaluation by the engine. It therefore should calculate once the base score, which will then be incrementally calculated. The `get_score` method should simply returns the current score. Then two function `push` and `pop` are defined to calculate a new score whenever a new moved is added or one removed, respectively. 

In [ ]:
class IncrementalEvaluator(Evaluator):

    def init(self, board: chess.Board):
        pass

    def get_score(self):
        pass

    def push(self, move: DetailedMove):
        pass

    def pop(self):
        pass

Next, we can implement the piece square evaluation incrementally. For this a new class `IncrementalPieceSquareEvaluator` is created that inherits `IncrementalEvaluator`. The constructor is the same as in the `PieceSquareEvaluator` except that it defines a list `scores` which be used to keep track of the incrementally calculated scores. Other methods like `heuristic` and helper functions are the same as well and therefore copied to this class.

In [ ]:
class IncrementalPieceSquareEvaluator(IncrementalEvaluator):

    def __init__(
        self,
        piece_values,
        piece_square_values,
        value_checkmate: int,
        value_draw: int
    ):
        super().__init__(value_checkmate, value_draw)
        self.piece_values = piece_values
        self.piece_square_values = piece_square_values
        self.scores = None


IncrementalPieceSquareEvaluator.heuristic = PieceSquareEvaluator.heuristic
IncrementalPieceSquareEvaluator._get_square_value = PieceSquareEvaluator._get_square_value
IncrementalPieceSquareEvaluator._get_piece_square_value = PieceSquareEvaluator._get_piece_square_value

The first function to implement is `init`. It will simply initalize the `scores` member with the base score, which is calculated by the `heuristic` method.´

In [ ]:
def init(self, board: chess.Board):
    self.scores = [self.heuristic(board)]


IncrementalPieceSquareEvaluator.init = init

As the scores are implementes as a list, the most recent score is the last one from the list and simply returned by the `get_score` method.

In [ ]:
def get_score(self):
    return self.scores[-1]


IncrementalPieceSquareEvaluator.get_score = get_score

The `push` method will take the move done as a parameter. It is assumed to be a `DetailedMove` and thus containing all needed data. The score is initalized to be the last calculated score. Then the value of the moved piece is subtracted and the value of the placed piece added. If there was a capture, then this value of this will be added to score as well. Then the calculated score will be put at the end of the `scors` list. The score needs to be negated as the player to move switched.

In [ ]:
def push(self, move: DetailedMove):
    score = self.scores[-1]

    score -= self._get_piece_square_value(
        move.movedPiece.piece, move.movedPiece.file, move.movedPiece.rank
    )

    score += self._get_piece_square_value(
        move.placedPiece.piece, move.placedPiece.file, move.placedPiece.rank
    )

    if move.has_capture():
        score += self._get_piece_square_value(
            move.capturedPiece.piece,
            move.capturedPiece.file,
            move.capturedPiece.rank
        )

    self.scores.append(-1 * score)


IncrementalPieceSquareEvaluator.push = push

The last missing function is `pop`, which simply removes the last score from the list. As one can see the pop function is very simple and efficient because the previous scores were saved in a list. It would also be possible to actually calculate the previous score, but that would be less efficient.

In [ ]:
def pop(self):
    self.scores.pop()


IncrementalPieceSquareEvaluator.pop = pop

Similiar to before, an evalutor for the simplified evaluation function can be defined using the `IncrementalPieceSquareEvaluator` class.

In [ ]:
incremental_simplified_evaluator = IncrementalPieceSquareEvaluator(
    piece_values=SIMPLIFIED_EVALUATION_PIECE_VALUES,
    piece_square_values=SIMPLIFIED_EVALUATION_PIECE_SQUARE_VALUES,
    value_checkmate=SIMPLIFIED_EVALUATION_VALUE_CHECKMATE,
    value_draw=SIMPLIFIED_EVALUATION_VALUE_DRAW
)

Our last engine `IterativeAlphaBetaCached` can use this evaluator already, but it won't make use of the new methods, to actually do the incremental evaluation. We therefore inherit as new class `IncrementalIterativeAlphaBetaCached` that not only requires an evluator of type `Evaluator`, but more specifically of type `IncrementalEvaluator`

In [ ]:
class IncrementalIterativeAlphaBetaCached(IterativeAlphaBetaCached):

    def __init__(
        self, evaluator: IncrementalEvaluator, max_look_ahead_depth: int
    ):
        super().__init__(evaluator, max_look_ahead_depth)

The `_evaluate_move` methods needs one additional line `self.evaluator.init(board)` to initalize the base score after the move was pushed to the board.

In [ ]:
from converted_notebooks.s04_engine_interface import ScoredMove


def _evaluate_move(self, board: chess.Board, move: chess.Move, depth: int):
    board.push(move)
    self.evaluator.init(board)

    score = self._value(
        board,
        depth - 1,
        -self.evaluator.value_checkmate,
        self.evaluator.value_checkmate
    )
    score *= self.PLAYER_MULTIPLIER[board.turn]
    board.pop()
    return ScoredMove(score=score, move=move)


IncrementalIterativeAlphaBetaCached._evaluate_move = _evaluate_move

The `value` method is similary almost the same as before with small adjustments. First the score returned at a depth of zero doesn't need to be calculated anymore, but can be returned with the `get_score` method. Additionally whenever a move is pushed to the board the `push` method of the evaluator needs to be called as well and the same for the `pop` method.

In [ ]:
from converted_notebooks.s11_iterative_deepening import cache_alpha_beta


@cache_alpha_beta
def _value(self, board: chess.Board, depth: int, alpha: int, beta: int) -> int:
    if (score := self.evaluator.evaluate(board)) is not None:
        return score
    if depth == 0:
        return self.evaluator.get_score()

    for move in board.legal_moves:
        detailedMove = DetailedMove(board, move)

        self.evaluator.push(detailedMove)
        board.push(move)
        value = -1 * self._value(board, depth - 1, -beta, -alpha)
        self.evaluator.pop()
        board.pop()

        if value >= beta:
            return value
        alpha = max(alpha, value)

    return alpha


IncrementalIterativeAlphaBetaCached._value = _value

We can now test the new defined engine. In this case the result can be again compared to the previous one as the incremental calculation will not change the score.

In [ ]:
import random
from converted_notebooks.s09_minimax_engine import middlegame_board
from converted_notebooks.s11_iterative_deepening import IterativeAlphaBetaCached

random.seed(42)

engine = IncrementalIterativeAlphaBetaCached(
    evaluator=incremental_simplified_evaluator, max_look_ahead_depth=3
)
result_incremental_simplified_evaluation = engine.analyse(middlegame_board)

assert result_incremental_simplified_evaluation == result_simplified_evaluation

An analyzation of the `_value` function with the help of a line profiler, shows that the additions only need 20% of the total time. This is a huge improvement compared to the 50% of the non incremental implementation.

In [ ]:
%load_ext line_profiler
random.seed(42)

engine = IncrementalIterativeAlphaBetaCached(
    evaluator=incremental_simplified_evaluator, max_look_ahead_depth=4
)
%lprun -f IncrementalIterativeAlphaBetaCached._value.__wrapped__ engine.analyse(middlegame_board)